In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4.Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── 5.без клауз, включающих местоимения.txt
│   ├── 6.без слов функциональных.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   ├── Второй_жанр без клауз, включающих местоимения.txt
│   ├── Второй_жанр без слов функциональных.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 1е_Изъ

In [2]:
import json

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

In [ ]:
import funciones
from utils import train_wrapper
import warnings
import os
import torch
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from contextlib import redirect_stderr
import multiprocessing as mp
import numpy as np
#import wandb
import nbformat
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import classification_report, confusion_matrix
import gc
import re
import nltk
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW
from razdel import sentenize
import numpy as np

# Suprimir warnings específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
mp.set_start_method('spawn', force=True)


import json


def convert_numpy(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
        return obj.item()
    return obj

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False, default=convert_numpy)

        
def create_custom_config(model_name, model_type, dataset):
    """Crea configuración de entrenamiento adaptativa"""
    common_config = {
#         'dataset_name':dataset.get('name', ''),
        'model_name': model_name,
        'model_type': model_type,
        'num_repeats': 6,
        'test_size': 0.2,
        'threshold': 0.5
    }

    # Configuración específica para GPT
#     if model_type == 'gpt':
#         return funciones.TrainingConfig(
#             **common_config,
#             max_length=52,
#             batch_size=64,
#             epochs=6,
#             learning_rate=1e-5
#         )

#         # Configuración por nombre de dataset
#         dataset_name = dataset.get('name', '')
#         if any(x in dataset_name for x in ['1', '2', '3', '4']):
#             return funciones.TrainingConfig(
#                 **common_config,
#                 max_length=60,
#                 batch_size=128,
#                 epochs=6,
#                 learning_rate=5e-5
#             )


    # Configuraciones basadas en frecuencia
    freq_threshold = dataset.get('freq_threshold')
    if freq_threshold in [100, 49, 29, 9, 5, 3]:
        configs = {
            100: (52, 16, 6, 2e-5),
            49: (60, 16, 6, 3e-5),
            29: (51, 16, 6, 3e-5),
            9: (45, 16, 6, 4e-5),
            5: (150, 16, 6, 5e-5),
            3: (150, 16, 6, 5e-5)
        }
        max_len, batch, epochs, lr = configs[freq_threshold]
        return funciones.TrainingConfig(
            **common_config,
            max_length=max_len,
            batch_size=batch,
            epochs=epochs,
            learning_rate=lr
        )

    # Configuración por nombre de dataset
    dataset_name = dataset.get('name', '')
    for x in ['1', '2', '3', '4', '5', '6']:
        if x in dataset_name:
            configs = {
                '1': (60, 16, 6, 5e-5),
                '2': (60, 16, 6, 5e-5),
                '3': (60, 16, 6, 5e-5),
                '4': (60, 16, 6, 5e-5),
                '5': (60, 16, 6, 5e-5),
                '6': (60, 16, 6, 5e-5)
            }
            max_len, batch, epochs, lr = configs[x]
            return funciones.TrainingConfig(
                **common_config,
                max_length=max_len,
                batch_size=batch,
                epochs=epochs,
                learning_rate=lr
            )


    # Configuración por defecto
    return funciones.TrainingConfig(
        **common_config,
        max_length=60,
        batch_size=32,
        epochs=4,
        learning_rate=3e-5
    )

models = [
    {'model':'DeepPavlov/rubert-base-cased',
     'name':'DeepPavlov-rubert-base',
     'type': 'bert'}, # Modelo original (ruso)
    {'model':'bert-base-multilingual-cased',
     'name':'BERT multilingual',
     'type': 'bert'}, # BERT multilingüe
    {'model':'distilbert-base-multilingual-cased',
     'name':'distilbert-base-multilingual',
     'type': 'bert'}, # Versión ligera de BERT multilingüe
    {'model':'roberta-base',
     'name':'roberta-base',
     'type': 'bert'}, # RoBERTa (principalmente inglés)

    {'model':'xlnet-base-cased',
     'name':'XLNet Base',
     'type': 'bert'}, # XLNet (multilingüe efectivo)
    {'model':'xlm-roberta-base',
     'name':'XLM-RoBERTa Base',
     'type': 'bert'}, # XLM-RoBERTa (multilingüe)
#     {'model':'google/rembert',
#      'name':'RemBERT',
#      'type': 'bert'}, # RemBERT (énfasis en lenguajes individuales)

    {'model': 'gpt2',
     'name': 'gpt2',
     'type': 'gpt'},  # GPT model
    {'model': 'facebook/opt-125m',
     'name': 'facebook/opt-125m',
     'type': 'gpt'},  # GPT model
    {'model': 'sberbank-ai/rugpt3small_based_on_gpt2',
     'name': 'rugpt3small',
     'type': 'gpt'}, # GPT model (ruso)
    {'model':'facebook/mbart-large-cc25',
     'name':'mBART Large',
     'type': 'gpt'} # mBART (generativo, multilingüe)
    ]
# [
#         {'model':'DeepPavlov/rubert-base-cased',
#          'name':'DeepPavlov-rubert-base',
#          'type': 'bert'},# Modelo original
#         {'model':'bert-base-multilingual-cased',
#          'name':'BERT multilingual',
#          'type': 'bert'},# BERT multilingüe
#         {'model':'distilbert-base-multilingual-cased',
#          'name':'distilbert-base-multilingual',
#          'type': 'bert'},# Versión ligera de BERT
#         {'model':'roberta-base',
#          'name':'roberta-base', 
#         'type': 'bert'}, # RoBERTa 
#         {'model': 'gpt2',
#          'name': 'gpt2',
#          'type': 'gpt'},  #  GPT model
#         {'model': 'facebook/opt-125m',
#          'name': 'facebook',
#          'type': 'gpt'},  #  GPT model
#         {'model': 'sberbank-ai/rugpt3small_based_on_gpt2',
#          'name': 'rugpt3',
#          'type': 'gpt'}
#     ]

datasets = [
    #deberia adicionar el set original?
        {
            'path1': '../dataset/сокращение по частотности/1б_Изъяты лексемы с частотой выше 100.txt',
            'path2': '../dataset/сокращение по частотности/2б_Изъяты лексемы с частотой выше 100.txt',
            'name': 'Изъяты лексемы с частотой выше 100',
            'type': 'freq',
            'freq': 100
        },
        {
            'path1': '../dataset/сокращение по частотности/1в_Изъяты лексемы с частотой выше 49.txt',
            'path2': '../dataset/сокращение по частотности/2в_Изъяты лексемы с частотой выше 49.txt',
            'name': 'Изъяты лексемы с частотой выше 49',
            'type': 'freq',
            'freq': 49
        },
        {
            'path1': '../dataset/сокращение по частотности/1г_Изъяты лексемы с частотой выше 29.txt',
            'path2': '../dataset/сокращение по частотности/2г_Изъяты лексемы с частотой выше 29.txt',
            'name': 'Изъяты лексемы с частотой выше 29',
            'type': 'freq',
            'freq': 29
        },
        {
            'path1': '../dataset/сокращение по частотности/1д_Изъяты лексемы с частотой выше 9.txt',
            'path2': '../dataset/сокращение по частотности/2д_Изъяты лексемы с частотой выше 9.txt',
            'name': 'Изъяты лексемы с частотой выше 9',
            'type': 'freq',
            'freq': 9
        },
        {
            'path1': '../dataset/сокращение по частотности/1е_Изъяты лексемы с частотой выше 5.txt',
            'path2': '../dataset/сокращение по частотности/2е_Изъяты лексемы с частотой выше 5.txt',
            'name': 'Изъяты лексемы с частотой выше 5',
            'type': 'freq',
            'freq': 5
        },
        {
            'path1': '../dataset/сокращение по частотности/1ё_Изъяты лексемы с частотой выше 3.txt',
            'path2': '../dataset/сокращение по частотности/2ё_Изъяты лексемы с частотой выше 3.txt',
            'name': 'Изъяты лексемы с частотой выше 3',
            'type': 'freq',
            'freq': 3
        },

            {
            'path1': '../dataset/Сокращение по частям речи/Без прилагательных первый жанр.txt',
            'path2': '../dataset/Сокращение по частям речи/Без прилагательных второй жанр.txt',
            'name': 'Без прилагательных первый-второй жанр',#litle correction
            'type': 'pos',
            'freq': None
            },
          {
            'path1': '../dataset/Сокращение по частям речи/1.Первый жанр исходная выборка.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '1.Первый жанр исходная выборка',# este el el orignal
            'type': 'pos',
            'freq': None

        },
        {
            'path1': '../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '2.Первый жанр без клауз, включающих наречия',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/3.Первый жанр без клауз, включающих глаголы.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '3.Первый жанр без клауз, включающих глаголы',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/4.Первый жанр без клауз, включающих глаголы и наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '4.Первый жанр без клауз, включающих глаголы и наречия',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/5.без клауз, включающих местоимения.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '5.без клауз, включающих местоимения.txt',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/6.без слов функциональных.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '6.без слов функциональных.txt',
            'type': 'pos',
            'freq': None
        },
    
    ]



russian_templates = [
    "Этот литературный фрагмент принадлежит к жанру {}.",
    "Данный текст является примером жанра {}.",
    "Стилистические особенности указывают на жанр {}.",
    "Жанровая принадлежность этого текста - {}.",
    "Эксперты классифицируют этот текст как жанр {}.",
    "Это типичный пример жанра {} в русской литературе.",
    "По ключевым характеристикам это текст жанра {}.",
    "Данное произведение относится к направлению {}.",
    "Анализ содержания позволяет отнести текст к жанру {}.",
    "Этот отрывок характерен для жанра {}."
]

def main():

    results = []
    for model in models:
        for dataset in datasets:
            config = create_custom_config(model['model'], model['type'], dataset)
            
            
            # Entrenar y limpiar memoria
            torch.cuda.empty_cache()
            result = funciones.train_and_evaluate_dataset(
                dataset['path1'],
                dataset['path2'],
                config,
                dataset['name'],
                dataset['type']
                
            )
            results.append(result) 
#             if model['type'] = 'gpt':
                
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
        
    
    save_results(results, 'resultados_Step_8.json') 
    
    
    

    
if __name__ == "__main__":
    main()

2025-04-23 13:02:44.193920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


train_Изъяты лексемы с частотой выше 100.csv
test_Изъяты лексемы с частотой выше 100.csv
Train labels sample: tensor([0, 0, 0, 0, 1]), Shape: torch.Size([864])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([195])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.13408770542850512
Epoch 2, Loss: 0.031455091472404696
Epoch 3, Loss: 0.0025195426995331355
Epoch 4, Loss: 0.0005683445762972244
Epoch 5, Loss: 0.00036313768456845236
Epoch 6, Loss: 0.00027049187364927666
all_preds shape: (195,)
all_labels shape: (195,)
all_probs shape: (195, 2)
Train labels sample: tensor([1, 0, 1, 0, 0]), Shape: torch.Size([864])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([198])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.15018639346170756
Epoch 2, Loss: 0.02072213014104852
Epoch 3, Loss: 0.001745199996986668
Epoch 4, Loss: 0.0005900748699248113
Epoch 5, Loss: 0.000363906

Epoch 1, Loss: 0.1718068239363757
Epoch 2, Loss: 0.05606303099881519
Epoch 3, Loss: 0.09510417614682493
Epoch 4, Loss: 0.03139557075602087
Epoch 5, Loss: 0.005103297860742631
Epoch 6, Loss: 0.0011785627766089006
all_preds shape: (211,)
all_labels shape: (211,)
all_probs shape: (211, 2)
Train labels sample: tensor([0, 0, 0, 1, 0]), Shape: torch.Size([869])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([209])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.1541359152814204
Epoch 2, Loss: 0.02884636555417356
Epoch 3, Loss: 0.001839271269273013
Epoch 4, Loss: 0.0005720990455963395
Epoch 5, Loss: 0.0003710168318568983
Epoch 6, Loss: 0.0002762339789610864
all_preds shape: (209,)
all_labels shape: (209,)
all_probs shape: (209, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([869])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([203])
##########################DeepPavlov/rubert-base-cased---b

all_preds shape: (732,)
all_labels shape: (732,)
all_probs shape: (732, 2)
Train labels sample: tensor([1, 1, 0, 0, 0]), Shape: torch.Size([90])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([739])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.5174157619476318
Epoch 2, Loss: 0.1163807703802983
Epoch 3, Loss: 0.017386416904628277
Epoch 4, Loss: 0.004886568562748532
Epoch 5, Loss: 0.0021773793074923256
Epoch 6, Loss: 0.0014313068822957575
all_preds shape: (739,)
all_labels shape: (739,)
all_probs shape: (739, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([90])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([718])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.482797568043073
Epoch 2, Loss: 0.16951334600647291
Epoch 3, Loss: 0.06808806800593932
Epoch 4, Loss: 0.027984825273354847
Epoch 5, Loss: 0.005579931118215124
Epoch 6, Loss: 0.

##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.11084152314473282
Epoch 2, Loss: 0.022410310739227994
Epoch 3, Loss: 0.04867012955841016
Epoch 4, Loss: 0.056985179262913084
Epoch 5, Loss: 0.03266947713511234
Epoch 6, Loss: 0.01883034180765125
all_preds shape: (174,)
all_labels shape: (174,)
all_probs shape: (174, 2)
Train labels sample: tensor([0, 0, 0, 0, 1]), Shape: torch.Size([876])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([170])
##########################DeepPavlov/rubert-base-cased---bert####################
Epoch 1, Loss: 0.14496760451319543
Epoch 2, Loss: 0.10191575576635924
Epoch 3, Loss: 0.025852628183466467
Epoch 4, Loss: 0.00232385948871855
Epoch 5, Loss: 0.0008716870956546204
Epoch 6, Loss: 0.0005625518319323997
all_preds shape: (170,)
all_labels shape: (170,)
all_probs shape: (170, 2)
Train labels sample: tensor([0, 0, 0, 1, 0]), Shape: torch.Size([876])
Test labels sample: tensor([0, 0, 0, 0, 0]), Sh

...

##########################distilbert-base-multilingual-cased---bert####################
Epoch 1, Loss: 0.7193352867256512
Epoch 2, Loss: 0.6712603059681979
Epoch 3, Loss: 0.6776172388683666
Epoch 4, Loss: 0.6765077385035428
Epoch 5, Loss: 0.425696587562561
Epoch 6, Loss: 0.3471831906925548
all_preds shape: (167,)
all_labels shape: (167,)
all_probs shape: (167, 2)
Train labels sample: tensor([1, 0, 0, 1, 0]), Shape: torch.Size([880])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([180])
##########################distilbert-base-multilingual-cased---bert####################
Epoch 1, Loss: 0.7171259349042719
Epoch 2, Loss: 0.6865696473555132
Epoch 3, Loss: 0.6789821863174439
Epoch 4, Loss: 0.6739152052185752
Epoch 5, Loss: 0.43497487841682003
Epoch 6, Loss: 0.11483573841608384
all_preds shape: (180,)
all_labels shape: (180,)
all_probs shape: (180, 2)
train_3.Первый жанр без клауз, включающих глаголы.csv
test_3.Первый жанр без клауз, включающих глаголы.csv
Train labels samp

Epoch 5, Loss: 0.3641974981184359
Epoch 6, Loss: 0.35265219818662713
all_preds shape: (181,)
all_labels shape: (181,)
all_probs shape: (181, 2)
Train labels sample: tensor([1, 0, 0, 1, 0]), Shape: torch.Size([858])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([189])
##########################distilbert-base-multilingual-cased---bert####################
Epoch 1, Loss: 0.6323789164975837
Epoch 2, Loss: 0.248795852297917
Epoch 3, Loss: 0.13469804476739633
Epoch 4, Loss: 0.08884060495808997
Epoch 5, Loss: 0.05162642190784768
Epoch 6, Loss: 0.03460912036071359
all_preds shape: (189,)
all_labels shape: (189,)
all_probs shape: (189, 2)
Train labels sample: tensor([0, 1, 0, 0, 0]), Shape: torch.Size([858])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([178])
##########################distilbert-base-multilingual-cased---bert####################
Epoch 1, Loss: 0.7141429031336749
Epoch 2, Loss: 0.3868784810916555
Epoch 3, Loss: 0.10757636948902574
Epoch 4, Loss

Epoch 6, Loss: 0.6780258523566383
all_preds shape: (189,)
all_labels shape: (189,)
all_probs shape: (189, 2)
train_Изъяты лексемы с частотой выше 49.csv
test_Изъяты лексемы с частотой выше 49.csv
Train labels sample: tensor([1, 1, 1, 0, 0]), Shape: torch.Size([916])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([195])
##########################roberta-base---bert####################
Epoch 1, Loss: 0.6781338401909532
Epoch 2, Loss: 0.6504219960549782
Epoch 3, Loss: 0.6540960494814247
Epoch 4, Loss: 0.6500619115500614
Epoch 5, Loss: 0.662700323195293
Epoch 6, Loss: 0.6500117624628132
all_preds shape: (195,)
all_labels shape: (195,)
all_probs shape: (195, 2)
Train labels sample: tensor([0, 0, 1, 0, 0]), Shape: torch.Size([916])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([187])
##########################roberta-base---bert####################
Epoch 1, Loss: 0.6943585276603699
Epoch 2, Loss: 0.6771119252361101
Epoch 3, Loss: 0.6600651078183075
Epoch 4, L

all_preds shape: (278,)
all_labels shape: (278,)
all_probs shape: (278, 2)
Train labels sample: tensor([0, 0, 1, 1, 0]), Shape: torch.Size([773])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([273])
##########################roberta-base---bert####################
Epoch 1, Loss: 0.7128013753890992
Epoch 2, Loss: 0.6529311108589172
Epoch 3, Loss: 0.6551114296913148
Epoch 4, Loss: 0.5644444066286087
all_preds shape: (273,)
all_labels shape: (273,)
all_probs shape: (273, 2)
train_Изъяты лексемы с частотой выше 5.csv
test_Изъяты лексемы с частотой выше 5.csv
Train labels sample: tensor([0, 1, 0, 0, 0]), Shape: torch.Size([99])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([524])
##########################roberta-base---bert####################
Epoch 1, Loss: 0.6290360923324313
Epoch 2, Loss: 0.645839090858187
Epoch 3, Loss: 0.6379866685186114
Epoch 4, Loss: 0.5795302774224963
Epoch 5, Loss: 0.5895850637129375
Epoch 6, Loss: 0.5733184899602618
all_preds sha

Train labels sample: tensor([0, 0, 0, 0, 1]), Shape: torch.Size([872])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([174])
##########################roberta-base---bert####################
Epoch 1, Loss: 0.6765413635543415
Epoch 2, Loss: 0.5852961997900691
Epoch 3, Loss: 0.488286737884794
Epoch 4, Loss: 0.4536716586777142
all_preds shape: (174,)
all_labels shape: (174,)
all_probs shape: (174, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([872])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([162])
##########################roberta-base---bert####################
Epoch 1, Loss: 0.6360957697033882
Epoch 2, Loss: 0.5490794777870178
Epoch 3, Loss: 0.4037646842854364
Epoch 4, Loss: 0.41505269919122967
all_preds shape: (162,)
all_labels shape: (162,)
all_probs shape: (162, 2)
train_1.Первый жанр исходная выборка.csv
test_1.Первый жанр исходная выборка.csv
Train labels sample: tensor([1, 0, 0, 1, 1]), Shape: torch.Size([912])
Test labels 

In [ ]:
#batch_size=128

In [ ]:
#display_results()

In [ ]:
import os
import IPython

# Reiniciar el kernel
IPython.display.display(IPython.display.Javascript("Jupyter.notebook.kernel.restart()"))

# Apagar el kernel después de reiniciar
os._exit(0)